# `demo.ipynb`
Demonstration of numpy, matplotlib, pandas, and xarray/geoviews

## numpy demo
Use Numpy to create a fake dataset of phytoplankton concentration across latitude and longitude.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

longitudes = np.linspace(-122.274, -121.727, 100)
latitudes = np.linspace(36.58,37.01, 100)

def fancy_function(x, y):
    """A 2D Gaussian."""
    return np.exp( 
        - ((x - longitudes.mean())**2 + (y - latitudes.mean())**2)) \
        / (latitudes.max() - latitudes.min()
    )

## pandas demo
Use pandas to label the dataset

In [ ]:
import pandas as pd

# for dataframe, we want each column to have a single quantity, like a spreadsheet.
data = np.zeros((len(latitudes) * len(longitudes), 3))

row = 0
for i in range(len(longitudes)):
    for j in range(len(latitudes)):
        data[row, 0] = longitudes[i]
        data[row, 1] = latitudes[j]
        data[row, 2] = fancy_function(longitudes[i], latitudes[j])

        row += 1

df = pd.DataFrame(data, columns=['Longitude', 'Latitude', 'plankton_concentration'])

df


## matplotlib demo
Plot an image of the generated dataset.

In [ ]:
fig, ax = plt.subplots()
hb = ax.hexbin(df.Longitude, df.Latitude, C=df.plankton_concentration, gridsize=20)
ax.axis([df.Longitude.min(), df.Longitude.max(), df.Latitude.min(), df.Latitude.max()])
ax.set_xlabel('Longitude ($^\circ$)')
ax.set_ylabel('Latitude ($^\circ$)')
fig.colorbar(hb, ax=ax, label='Plankton Per Planck Length')
fig.suptitle('Plankton Concentration Near MBARI')

In [ ]:
# Aside: "for" loops in Python can be kind of slow. With numpy, it is often faster to use 
# "vectorized" operations that use numpy's built-in iterators instead of for loops. However,
# you need to be very familiar with the different functions numpy provides: its "API". Here's
# how this looks for our toy example.

# Create a (100 X 100) array of x-coordinates
# and a (100 X 100) array of y-coordinates
# such that (XX[i,j], YY[i,j]) = (x_i, y_j)
YY, XX = np.meshgrid(latitudes, longitudes)

# Use ravel() to collapse each coordinate array into a 1D vector.
# Because the numpy operation in fancy_function are vectorized, they can quickly process
# these vectors in parallel (using numpy's C++ code).
# This line creates a (10_000,) element vector.
plankton_concentration = fancy_function(XX.ravel(), YY.ravel()).reshape(len(XX), len(YY))

# Collect the latitude data, longitude data, and plankton concentration into a single array.
data2 = np.stack([XX.ravel(), YY.ravel(), plankton_concentration.ravel()]).T

# The result you get is the same either way!
assert(np.allclose(data, data2))

# The vectorized implementation is 127 us, and the for-loop implementation is 139 ms,
# around a ~1000x speedup. (I used the %%timeit cell magic to test.)

## geoviews demo
Use geoviews to plot the data on a fancy interactive globe.

In [ ]:
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
gv.extension('bokeh', 'matplotlib')

In [ ]:
import xarray as xr
xrds = xr.Dataset.from_dataframe(df.set_index(['Longitude', 'Latitude']))

In [ ]:
ds = gv.Dataset(xrds)

In [ ]:
ds.to(gv.Image, ['Longitude', 'Latitude'], 'plankton_concentration')

In [ ]:
ds.plot()

In [ ]:
from geoviews import opts
(ds.to.image(['Longitude', 'Latitude']) * gf.coastline).opts(
    opts.Image(projection=crs.Geostationary(), cmap='Greens', xaxis=None, yaxis=None))

In [ ]:
(gv.Polygons(ds) * gf.coastline).opts(
    width=600, height=400, tools=['hover'], infer_projection=True)